In [ ]:
import pandas as pd
import numpy as np
import datetime
import os

# Correct path formatting (use a raw string or replace \ with /)
folder_path = r'C:\Users\skhadijah\Downloads\FAH\ANALYSIS'
# Alternatively: folder_path = 'C:/Users/skhadijah/Downloads/FAH/ANALYSIS'

data_frames = []

for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file)
        try:
            df = pd.read_excel(file_path)
            data_frames.append(df)
        except Exception as e:
            print(f"Failed to read {file_path}: {e}")

# Concatenate all data frames into one, handling potential issues with non-aligned columns
merged_df = pd.concat(data_frames, ignore_index=True, sort=False)

# List of columns you're interested in
selected_columns = [
    'Accounting Date', 'Accounting Event', 'Accounting Class','Enter Currency', 'Entered DR', 'Entered CR',
    'Accounted DR', 'Accounted CR', 'Source Accounting Event',
    'Accounting Line Type', 'Customer Number', 'Customer Account Number', 'Transaction Number',
    'Credit Debit Indicator', 'Posting Date'
]


# Assuming 'cust' is your DataFrame
gl = merged_df[selected_columns]

#Save file
gl.to_csv('C:/Users/skhadijah/Downloads/full_gl.csv', index=False)

gl.head(5)


In [ ]:
gl.info()

In [ ]:
#Aggregate for daily

# Ensure the 'Accounting Date' is a datetime type
gl['Accounting Date'] = pd.to_datetime(gl['Accounting Date'])

#Filter all the data to select "Accounting Line Type" for CUSTOMER_CMSA and CUSTOMER_CMSA_SAVINGS_POT
gl = gl[gl['Accounting Line Type'].isin(['CUSTOMER_CMSA', 'CUSTOMER_CMSA_SAVINGS_POT'])]

# Add a column for debit (money out) and credit (money in) counts
# Assign 1 to every transaction for counting purposes
gl['Debit_Count'] = (gl['Entered DR'] > 0).astype(int)
gl['Credit_Count'] = (gl['Entered CR'] > 0).astype(int)

# Group by 'Customer Account Number' and 'Accounting Date' (daily as an example)
grouped = gl.groupby(['Customer Account Number', pd.Grouper(key='Accounting Date', freq='D')])

# Aggregate transactions
daily_aggregated_transactions = grouped.agg(
    count_debit=pd.NamedAgg(column='Debit_Count', aggfunc='sum'),
    amount_debit=pd.NamedAgg(column='Entered DR', aggfunc='sum'),
    count_credit=pd.NamedAgg(column='Credit_Count', aggfunc='sum'),
    amount_credit=pd.NamedAgg(column='Entered CR', aggfunc='sum')
)

# Calculate daily balance for each customer account (cumulative credits - cumulative debits)
daily_aggregated_transactions['balance'] = daily_aggregated_transactions.groupby(level=0).cumsum().eval('amount_credit - amount_debit')

# Reset index for daily aggregated transactions
daily_aggregated_transactions = daily_aggregated_transactions.reset_index()

#Save file
daily_aggregated_transactions.to_csv('C:/Users/skhadijah/Downloads/customer_trend/daily_agg.csv', index=False)

# Display the first few rows to check
daily_aggregated_transactions.head()


In [ ]:
#Aggregate Weekly

# Group by 'Customer Account Number' and 'Accounting Date' (weekly)
weekly_grouped = gl.groupby(['Customer Account Number', pd.Grouper(key='Accounting Date', freq='W')])

#Filter all the data to select "Accounting Line Type" for CUSTOMER_CMSA and CUSTOMER_CMSA_SAVINGS_POT
gl = gl[gl['Accounting Line Type'].isin(['CUSTOMER_CMSA', 'CUSTOMER_CMSA_SAVINGS_POT'])]

# Aggregate transactions for weekly data
weekly_aggregated_transactions = weekly_grouped.agg(
    count_debit=pd.NamedAgg(column='Debit_Count', aggfunc='sum'),
    amount_debit=pd.NamedAgg(column='Entered DR', aggfunc='sum'),
    count_credit=pd.NamedAgg(column='Credit_Count', aggfunc='sum'),
    amount_credit=pd.NamedAgg(column='Entered CR', aggfunc='sum')
)

# Calculate weekly balance for each customer account (cumulative credits - cumulative debits)
weekly_aggregated_transactions['balance'] = weekly_aggregated_transactions.groupby(level=0).cumsum().eval('amount_credit - amount_debit')

# Reset index for daily aggregated transactions
weekly_aggregated_transactions = weekly_aggregated_transactions.reset_index()

#Save file
weekly_aggregated_transactions.to_csv('C:/Users/skhadijah/Downloads/customer_trend/weekly_agg.csv', index=False)

# Display the first few rows to check weekly data
weekly_aggregated_transactions.head()


In [ ]:
#Monthly Aggregation

# Group by 'Customer Account Number' and 'Accounting Date' (monthly)
monthly_grouped = gl.groupby(['Customer Account Number', pd.Grouper(key='Accounting Date', freq='M')])

#Filter all the data to select "Accounting Line Type" for CUSTOMER_CMSA and CUSTOMER_CMSA_SAVINGS_POT
gl = gl[gl['Accounting Line Type'].isin(['CUSTOMER_CMSA', 'CUSTOMER_CMSA_SAVINGS_POT'])]

# Aggregate transactions for monthly data
monthly_aggregated_transactions = monthly_grouped.agg(
    count_debit=pd.NamedAgg(column='Debit_Count', aggfunc='sum'),
    amount_debit=pd.NamedAgg(column='Entered DR', aggfunc='sum'),
    count_credit=pd.NamedAgg(column='Credit_Count', aggfunc='sum'),
    amount_credit=pd.NamedAgg(column='Entered CR', aggfunc='sum')
)

# Calculate monthly balance for each customer account (cumulative credits - cumulative debits)
monthly_aggregated_transactions['Net Flow'] = monthly_aggregated_transactions.groupby(level=0).cumsum().eval('amount_credit - amount_debit')

# Reset index for daily aggregated transactions
monthly_aggregated_transactions = monthly_aggregated_transactions.reset_index()

#Save file
monthly_aggregated_transactions.to_csv('C:/Users/skhadijah/Downloads/customer_trend/monthly_agg.csv', index=False)

# Display the first few rows to check monthly data
monthly_aggregated_transactions.head()


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# Assuming 'gl' is your DataFrame containing transaction data

# Step 1: Prepare the DataFrame
gl['Accounting Date'] = pd.to_datetime(gl['Accounting Date'])
gl = gl[gl['Accounting Line Type'].isin(['CUSTOMER_CMSA', 'CUSTOMER_CMSA_SAVINGS_POT'])]
gl['Debit_Count'] = (gl['Entered DR'] > 0).astype(int)
gl['Credit_Count'] = (gl['Entered CR'] > 0).astype(int)
# gl = gl[gl['Accounting Date'] >= pd.Timestamp('2022-12-01')]
# gl = gl[gl['Accounting Date'] <= pd.Timestamp('2024-03-31')]
# gl = gl[(gl['Accounting Date'] >= pd.Timestamp('2022-01-01')) & (gl['Accounting Date'] <= pd.Timestamp('2024-03-31'))]


# Aggregate daily transactions
grouped = gl.groupby(['Customer Account Number', pd.Grouper(key='Accounting Date', freq='D')])
daily_aggregated_transactions = grouped.agg(
    count_debit=pd.NamedAgg(column='Debit_Count', aggfunc='sum'),
    amount_debit=pd.NamedAgg(column='Entered DR', aggfunc='sum'),
    count_credit=pd.NamedAgg(column='Credit_Count', aggfunc='sum'),
    amount_credit=pd.NamedAgg(column='Entered CR', aggfunc='sum')
)

# Calculate daily balance
daily_aggregated_transactions['balance'] = daily_aggregated_transactions.groupby(level=0).cumsum().eval('amount_credit - amount_debit')

# Step 2: Calculate RFM Metrics
now = datetime.now()  # Consider using a specific reference date for analysis consistency
gl['Net Amount'] = gl['Entered CR'] - gl['Entered DR']
rfm = gl.groupby('Customer Account Number').agg(
    Recency=('Accounting Date', lambda x: (now - x.max()).days),
    Frequency=('Customer Account Number', 'count'),
    Monetary=('Net Amount', 'sum')
)

# Step 3: Score and Segment (with duplicate bin edge handling)
quantiles = rfm.quantile(q=[0.25, 0.5, 0.75]).to_dict()

# Ensure bins are unique by adjusting them slightly if duplicates are found
bins_recency = sorted(set([-np.inf, quantiles['Recency'][0.25], quantiles['Recency'][0.5], quantiles['Recency'][0.75], np.inf]))
bins_frequency = sorted(set([-np.inf, quantiles['Frequency'][0.25], quantiles['Frequency'][0.5], quantiles['Frequency'][0.75], np.inf]))
bins_monetary = sorted(set([-np.inf, quantiles['Monetary'][0.25], quantiles['Monetary'][0.5], quantiles['Monetary'][0.75], np.inf]))

# Apply scoring, adjusting label counts based on the number of unique bins
rfm['R_Score'] = pd.cut(rfm['Recency'], bins=bins_recency, labels=range(len(bins_recency)-1, 0, -1), right=False)
rfm['F_Score'] = pd.cut(rfm['Frequency'], bins=bins_frequency, labels=range(1, len(bins_frequency)), right=False)
rfm['M_Score'] = pd.cut(rfm['Monetary'], bins=bins_monetary, labels=range(1, len(bins_monetary)), right=False)

def assign_segment(row):
    if row['R_Score'] > 2 and row['F_Score'] > 2:
        return 'Loyal Customers'
    elif row['R_Score'] > 2 and row['F_Score'] <= 2:
        return 'Potential Loyalist'
    elif row['R_Score'] > 2 and row['F_Score'] == 1:
        return 'Recent Users'
    elif row['R_Score'] == 1 and row['F_Score'] > 2:
        return 'Needs Attention'
    elif row['R_Score'] == 1 and row['F_Score'] <= 2:
        return 'About To Sleep'
    elif row['M_Score'] == 1:
        return 'Price Sensitive'
    elif row['F_Score'] == 1 and row['M_Score'] > 2:
        return "Can't Lose Them"
    elif row['R_Score'] == 1 and row['F_Score'] == 1:
        return 'Hibernating'
    else:
        return 'Promising'
    

# # Scoring the RFM metrics
# rfm['F_Score'] = pd.qcut(rfm['Frequency'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')
# rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')
# rfm['R_Score'] = pd.qcut(rfm['Recency'].rank(method='first'), 5, labels=[5, 4, 3, 2, 1])

# def assign_segment(row):
#     # Convert quintile labels to integers for comparison
#     F_Score = int(row['F_Score'])
#     R_Score = int(row['R_Score'])
#     M_Score = int(row['M_Score'])

#     if F_Score == 5 and R_Score >= 2 and M_Score == 5:
#         return 'Loyal Customers (Champions)'
#     elif F_Score >= 3 and R_Score >= 2 and M_Score >= 3:
#         return 'Emerging Loyalists (Potential Loyalist)'
#     elif R_Score == 5 and F_Score <= 3 and M_Score in [2, 3]:
#         return 'New Enthusiasts (Recent Users)'
#     elif R_Score >= 2 and F_Score <= 3 and M_Score == 2:
#         return 'Inconsistent Savers (Promising)'
#     elif R_Score == 1 and F_Score in [3, 4] and M_Score in [2, 3]:
#         return 'Casual Bankers (Needs Attention)'
#     elif R_Score == 1 and F_Score <= 2 and M_Score == 5:
#         return 'Slipping Away (Can\'t Lose Them)'
#     elif R_Score <= 2 and F_Score <= 2 and M_Score <= 2:
#         return 'Hibernating'
#     elif F_Score in [1, 2] and R_Score in [1, 2] and M_Score in [1, 2]:
#         return 'About To Sleep'
#     elif F_Score in [1, 2] and M_Score in [1, 2]:
#         return 'Price Sensitive'
#     elif R_Score == 1 and F_Score == 1 and M_Score == 1:
#         return 'Lost'
#     else:
#         return 'Other'

# Apply the function to the RFM dataframe
rfm['Segment'] = rfm.apply(assign_segment, axis=1)

# Reset index for daily aggregated transactions
rfm = rfm.reset_index()

#Save file
rfm.to_csv('C:/Users/skhadijah/Downloads/customer_trend/rfm.csv', index=False)

# Display the results
rfm.head()


In [ ]:
#Merge customer data and RFM

import pandas as pd

# Load the customer data
file_path = 'C:/Users/skhadijah/Downloads/Customer-Ultimate-Balance-V2-20052024.csv'
cust_df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Select the variables you want to keep
selected_columns = [
    'customerId', 'tm_account_id', 'account_accountNumber', 'createdOn_x_x',
    'lastUpdate', 'overallStatus', 'fullname', 'age', 'placeOfBirth', 'gender',
    'nationality', 'type', 'device_os', 'device_versioning', 'device_model',
    'ledger_balance', 'balance_created_balance_denomination', 'total_credit',
    'total_debit', 'last_transaction_date', 'device_status',
    'preferences_pushNotificationsAllowed', 'preferences_marketingEmailFlag',
    'preferences_marketingPhoneCallFlag', 'preferences_marketingSmsFlag',
    'preferences_marketingPushFlag', 'account_status', 'offboardingStatus_string',
    'ftv_otherSideFullName', 'ftv_otherSideBankCode', 'verificationStatus',
    'status', 'number', 'openingTimestamp', 'activationTimestamp', 'employer',
    'employmentType', 'employmentSector', 'occupation', 'annualIncomeBracket',
    'residential_address_line1', 'residential_address_line2', 'residential_address_line3',
    'residential_address_line4', 'residential_address_line5', 'residential_address_country_code',
    'residential_address_city', 'residential_address_postal_code', 'residential_address_subdivision',
    'residential_address_subdivision_code', 'residential_address_type', 'mailing_address_line1',
    'mailing_address_line2', 'mailing_address_line3', 'mailing_address_line4', 'mailing_address_line5',
    'mailing_address_country_code', 'mailing_address_city', 'mailing_address_postal_code',
    'mailing_address_subdivision', 'mailing_address_subdivision_code', 'mailing_address_type',
    'mailing_address_createdOn', 'mailing_address_lastUpdated', 'mailing_address_smeId', 'maritalStatus',
    'ethnicity', 'Bumi', 'residencyStatus', 'entityType', 'accountSettingUpReasons_0',
    'accountSettingUpReasons_1', 'accountSettingUpReasons_2', 'accountSettingUpReasons_3', 'crr_score',
    'crr_rating', 'crr_timestamp', 'nameScreeningMatch', 'matchStatus', 'totalHits', 'totalBlacklistHits',
    'complyAdv_timestamp'
]

# Ensure that the column exists before selecting to avoid KeyError
cust_selected = cust_df[selected_columns]

# Merge the selected customer data with the RFM dataframe on the account number
# Assuming the RFM dataframe is named 'rfm' and the common identifier column is 'account_accountNumber' in 'cust_selected' and 'Customer Account Number' in 'rfm'
merge_df = pd.merge(rfm, cust_selected,  how='inner', left_on='Customer Account Number', right_on='account_accountNumber')

# Modify the 'ethnicity' and 'maritalStatus' columns based on the given mappings
merge_df['ethnicity'] = merge_df['ethnicity'].replace({'Cina': 'Chinese', 'Melayu': 'Malay'})
merge_df['maritalStatus'] = merge_df['maritalStatus'].replace({'Bujang': 'Single', 'Berkahwin': 'Married'})

#Save file
merge_df.to_csv('C:/Users/skhadijah/Downloads/customer_trend/rfm_cust.csv', index=False)

# Show the first few rows of the merged dataframe
merge_df.head()


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import scipy.stats as stats

# Assuming 'rfm' is your DataFrame and it's already loaded with 'Recency', 'Frequency', 'Monetary' columns

# Function to add a normal distribution curve to histograms
def add_normal_curve(fig, data):
    mean = np.mean(data)
    std = np.std(data)
    min_val, max_val = np.min(data), np.max(data)
    x = np.linspace(min_val, max_val, num=300)
    y = stats.norm.pdf(x, mean, std)
    y = y / max(y) * data.value_counts().max()  # Normalize to the height of the histogram
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='Normal fit', line=dict(color='black', width=2)))

# Plot for Recency
fig_recency = px.histogram(rfm, x='Recency',
                           title='Distribution of Recency',
                           labels={'Recency': 'Days since last transaction'},
                           nbins=50, 
                           color_discrete_sequence=['#636EFA'])
add_normal_curve(fig_recency, rfm['Recency'])
fig_recency.show()

# Plot for Frequency
fig_frequency = px.histogram(rfm, x='Frequency',
                             title='Distribution of Frequency',
                             labels={'Frequency': 'Number of transactions'},
                             nbins=50, 
                             color_discrete_sequence=['#EF553B'])
add_normal_curve(fig_frequency, rfm['Frequency'])
fig_frequency.show()

# Plot for Monetary
fig_monetary = px.histogram(rfm, x='Monetary',
                            title='Distribution of Monetary Value',
                            labels={'Monetary': 'Total spent'},
                            nbins=50, 
                            color_discrete_sequence=['#00CC96'])
add_normal_curve(fig_monetary, rfm['Monetary'])
fig_monetary.show()


In [ ]:
## Month on Month Balance

import pandas as pd

# Convert 'Accounting Date' to datetime format
gl['Accounting Date'] = pd.to_datetime(gl['Accounting Date'])

# Filter to include only relevant 'Accounting Line Type'
gl = gl[gl['Accounting Line Type'].isin(['CUSTOMER_CMSA', 'CUSTOMER_CMSA_SAVINGS_POT'])]

# Create count columns for debits and credits
gl['Debit_Count'] = (gl['Entered DR'] > 0).astype(int)
gl['Credit_Count'] = (gl['Entered CR'] > 0).astype(int)

# Group by 'Customer Account Number' and 'Accounting Date' with a monthly frequency
grouped = gl.groupby(['Customer Account Number', pd.Grouper(key='Accounting Date', freq='M')])

# Aggregate transactions monthly
monthly_aggregated_transactions = grouped.agg(
    count_debit=pd.NamedAgg(column='Debit_Count', aggfunc='sum'),
    amount_debit=pd.NamedAgg(column='Entered DR', aggfunc='sum'),
    count_credit=pd.NamedAgg(column='Credit_Count', aggfunc='sum'),
    amount_credit=pd.NamedAgg(column='Entered CR', aggfunc='sum')
)
# Calculate monthly balance for each customer account
monthly_aggregated_transactions['balance'] = monthly_aggregated_transactions['amount_credit'] - monthly_aggregated_transactions['amount_debit']

# Calculate month-on-month change
monthly_aggregated_transactions['mom_change'] = monthly_aggregated_transactions.groupby(level=0)['balance'].diff().fillna(0)

# Reset index
monthly_aggregated_transactions.reset_index(inplace=True)

# File path to save the CSV file
output_file_path = 'C:/Users/skhadijah/Downloads/customer_trend/monthly_balance_change.csv'
monthly_aggregated_transactions.to_csv(output_file_path, index=False)

print("File has been saved to:", output_file_path)
# Display the first few rows to check the output
monthly_aggregated_transactions.head()

In [ ]:
import pandas as pd
import numpy as np
import datetime
import os

# Assume `gl` is already prepared with the relevant data loaded as shown in the previous examples.

# Convert 'Accounting Date' to datetime format
gl['Accounting Date'] = pd.to_datetime(gl['Accounting Date'])

# Filter to include only relevant 'Accounting Line Type'
gl = gl[gl['Accounting Line Type'].isin(['CUSTOMER_CMSA', 'CUSTOMER_CMSA_SAVINGS_POT'])]

# Create count columns for debits and credits
gl['Debit_Count'] = (gl['Entered DR'] > 0).astype(int)
gl['Credit_Count'] = (gl['Entered CR'] > 0).astype(int)

# Group by 'Accounting Date' with a monthly frequency
grouped = gl.groupby(pd.Grouper(key='Accounting Date', freq='M'))

# Aggregate transactions monthly
monthly_aggregated_transactions = grouped.agg(
    count_debit=pd.NamedAgg(column='Debit_Count', aggfunc='sum'),
    amount_debit=pd.NamedAgg(column='Entered DR', aggfunc='sum'),
    count_credit=pd.NamedAgg(column='Credit_Count', aggfunc='sum'),
    amount_credit=pd.NamedAgg(column='Entered CR', aggfunc='sum')
)

# Calculate monthly balance for the bank
monthly_aggregated_transactions['balance'] = monthly_aggregated_transactions['amount_credit'] - monthly_aggregated_transactions['amount_debit']

# Calculate month-on-month change
monthly_aggregated_transactions['mom_change'] = monthly_aggregated_transactions['balance'].diff().fillna(0)

# Reset index to get 'Accounting Date' as a column
monthly_aggregated_transactions.reset_index(inplace=True)

# File path to save the CSV file
output_file_path = 'C:/Users/skhadijah/Downloads/customer_trend/monthly_balance_change_bankwide.csv'
monthly_aggregated_transactions.to_csv(output_file_path, index=False)

print("File has been saved to:", output_file_path)
# Display the first few rows to check the output
monthly_aggregated_transactions


In [ ]:
import pandas as pd
import numpy as np
import datetime
import os

# Assuming `gl` DataFrame is already prepared with the relevant data loaded.

# Convert 'Accounting Date' to datetime format
gl['Accounting Date'] = pd.to_datetime(gl['Accounting Date'])

# Filter to include only relevant 'Accounting Line Type'
gl = gl[gl['Accounting Line Type'].isin(['CUSTOMER_CMSA', 'CUSTOMER_CMSA_SAVINGS_POT'])]

# Adjust the 'Accounting Date' to start from 18th of each month
# Subtract days to set all dates to the previous month if before the 18th
gl['Adjusted Date'] = gl['Accounting Date'].apply(lambda x: x - pd.DateOffset(days=x.day-18) if x.day >= 18 else x - pd.DateOffset(months=1, days=x.day-18))

# Create count columns for debits and credits
gl['Debit_Count'] = (gl['Entered DR'] > 0).astype(int)
gl['Credit_Count'] = (gl['Entered CR'] > 0).astype(int)

# Group by 'Adjusted Date' with a monthly frequency starting from 18th
grouped = gl.groupby(pd.Grouper(key='Adjusted Date', freq='M'))

# Aggregate transactions monthly
monthly_aggregated_transactions = grouped.agg(
    count_debit=pd.NamedAgg(column='Debit_Count', aggfunc='sum'),
    amount_debit=pd.NamedAgg(column='Entered DR', aggfunc='sum'),
    count_credit=pd.NamedAgg(column='Credit_Count', aggfunc='sum'),
    amount_credit=pd.NamedAgg(column='Entered CR', aggfunc='sum')
)

# Calculate monthly balance for the bank
monthly_aggregated_transactions['balance'] = monthly_aggregated_transactions['amount_credit'] - monthly_aggregated_transactions['amount_debit']

# Calculate month-on-month change
monthly_aggregated_transactions['mom_change'] = monthly_aggregated_transactions['balance'].diff().fillna(0)

# Reset index to get 'Adjusted Date' as a column
monthly_aggregated_transactions.reset_index(inplace=True)

# File path to save the CSV file
output_file_path = 'C:/Users/skhadijah/Downloads/customer_trend/monthly_balance_change_bankwide-18th.csv'
monthly_aggregated_transactions.to_csv(output_file_path, index=False)

print("File has been saved to:", output_file_path)
# Display the first few rows to check the output
print(monthly_aggregated_transactions.head())
